## Imports

In [ ]:
%pip install Bio
from Bio.SeqUtils import IsoelectricPoint
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import statistics
!apt-get update
!apt-get install emboss
import pandas as pd
import numpy as np
import pandas as pd
import os
from Bio import SeqIO
from typing import Sequence
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, ReLU, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.activations import sigmoid
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler


## Testing out the ProtParam

In [2]:
# Create a protein sequence object
sequence = Seq("MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV")

# Calculate the statistics
stats = ProteinAnalysis(sequence)

# Print the results
count = stats.count_amino_acids()
percent = stats.get_amino_acids_percent()
print(stats.molecular_weight())
print(stats.aromaticity())
print(stats.instability_index())
sec_struc = stats.secondary_structure_fraction()  # [helix, turn, sheet]
print("%0.2f" % sec_struc[0])  # helix
print("%0.2f" % sec_struc[1])  # turn
print("%0.2f" % sec_struc[2])  # sheet
epsilon_prot = stats.molar_extinction_coefficient()  # [reduced, oxidized]
print(epsilon_prot[0])  # with reduced cysteines
charge = stats.charge_at_pH(10)
print(charge)
charge = stats.charge_at_pH(7)
print(charge)
charge = stats.charge_at_pH(4)
print(charge)
isoelectric = stats.isoelectric_point()
print(isoelectric)
gravy = stats.gravy()
print(gravy)
flex = stats.flexibility()
print(statistics.mean(flex))
print(stats.molecular_weight() / sum(count.values()))

tiny = percent["A"] + percent["C"] + percent["G"] + percent["S"] + percent["T"]  
small = percent["A"] + percent["C"] + percent["D"] + percent["G"] \
      + percent["N"] + percent["P"] + percent["S"] + percent["T"] + percent["V"]
aliphatic = percent["A"] + percent["I"] + percent["L"] + percent["V"]   
aromatic = percent["F"] + percent["H"] + percent["W"] + percent["Y"]   
non_polar = percent["A"] + percent["C"] + percent["F"] + percent["G"] + percent["I"] \
          + percent["L"] + percent["M"] + percent["P"] + percent["V"] + percent["W"] \
          + percent["Y"] 
polar = percent["D"] + percent["E"] + percent["H"] + percent["K"] + percent["N"] \
      + percent["Q"] + percent["R"] + percent["S"] + percent["T"] 
charged = percent["D"] + percent["E"] + percent["H"] + percent["K"] + percent["R"] 
basic = percent["H"] + percent["K"] + percent["R"]  
acidic = percent["D"] + percent["E"]   
ala = percent["A"] 
arg = percent["R"] 
asn = percent["N"] 
asp = percent["D"] 
cys = percent["C"] 

17103.1617
0.09868421052631579
41.980263157894726
0.28
0.26
0.25
17420
-12.785162430771866
0.9258119875149688
17.71008924714465
7.7224523544311525
-0.597368421052632
1.0058031968031969
112.52080065789474


## Importing Data

In [3]:
directory = '/content'
data = []

for file in os.listdir(directory):
    if file.endswith('.fa'):
        ix = 0
        for record in SeqIO.parse(os.path.join(directory, file), 'fasta'):
            sequence = str(record.seq)
            function = file.split('_')[0]
            index = f"{function}_{ix}"
            data.append({'Sequence': sequence, 'Class': function, 'Index': index})
            ix += 1

df = pd.DataFrame(data)
df.set_index('Index', inplace=True)


In [4]:
df

,Sequence,Class
Index,,
5_0,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...,5
5_1,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...,5
5_2,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...,5
5_3,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...,5
5_4,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...,5
...,...,...
4_1995,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...,4
4_1996,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...,4
4_1997,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...,4


### Prepare Binary for binary classification

In [5]:
df['Class'] = df['Class'].str.replace('non-enzyme', '8')


In [6]:
df['Binary'] = df['Class'].apply(lambda x: 1 if x in ["1", "2", "3", "4", "5", "6", "7"] else 0)
df

,Sequence,Class,Binary
Index,,,
5_0,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1
5_1,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1
5_2,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...,5,1
5_3,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...,5,1
5_4,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...,5,1
...,...,...,...
4_1995,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...,4,1
4_1996,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...,4,1
4_1997,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...,4,1


## Creating the stats table

In [7]:
df = df.reset_index()
df["Sequence"] = df["Sequence"].str.replace("X", "")
df["Sequence"] = df["Sequence"].str.replace("U", "C")

# Iterate through the rows of the DataFrame and create new objects
statistical = []
for i, row in df.iterrows():
    obj_stats = []
    stats = ProteinAnalysis(row["Sequence"])
    count = stats.count_amino_acids()
    percent = stats.get_amino_acids_percent()
    obj_stats.append(round(stats.molecular_weight(),2))
    obj_stats.append(round(stats.aromaticity(),2))
    obj_stats.append(round(stats.instability_index(),2))
    sec_struc = stats.secondary_structure_fraction()  # [helix, turn, sheet]
    obj_stats.append(round(sec_struc[0],2))
    obj_stats.append(round(sec_struc[1],2))
    obj_stats.append(round(sec_struc[2],2))

    obj_stats.append(round(stats.molar_extinction_coefficient()[0],2))
    obj_stats.append(round(stats.charge_at_pH(10),2))
    obj_stats.append(round(stats.charge_at_pH(7),2))
    obj_stats.append(round(stats.charge_at_pH(4),2))

    obj_stats.append(round(stats.isoelectric_point(),2))
    obj_stats.append(round(stats.gravy(),2))

    flex = stats.flexibility()

    obj_stats.append(round(statistics.mean(flex),2))
    obj_stats.append(round(stats.molecular_weight() / sum(count.values()),2))

    obj_stats.append(round(percent["A"] + percent["C"] + percent["G"] \
                         + percent["S"] + percent["T"],2))
    
    obj_stats.append(round(percent["A"] + percent["C"] + percent["D"] \
                         + percent["G"] + percent["N"] + percent["P"] \
                         + percent["S"] + percent["T"] + percent["V"],2))
    
    obj_stats.append(round( percent["A"] + percent["I"] + percent["L"] \
                          + percent["V"] ,2))

    obj_stats.append(round( percent["F"] + percent["H"] + percent["W"] \
                          + percent["Y"] ,2))
    
    obj_stats.append(round( percent["A"] + percent["C"] + percent["F"] \
                          + percent["G"] + percent["I"] + percent["L"] \
                          + percent["M"] + percent["P"] + percent["V"] \
                          + percent["W"] + percent["Y"],2))
    
    
    obj_stats.append(round( percent["D"] + percent["E"] + percent["H"] \
                          + percent["K"] + percent["N"] + percent["Q"] \
                          + percent["R"] + percent["S"] + percent["T"],2))

    obj_stats.append(round( percent["D"] + percent["E"] + percent["H"] \
                          + percent["K"] + percent["R"],2))
    
    obj_stats.append(round( percent["H"] + percent["K"] + percent["R"],2))

    obj_stats.append(round( percent["D"] + percent["E"],2))

    obj_stats.append(round( percent["A"],2))
    obj_stats.append(round( percent["R"],2))
    obj_stats.append(round( percent["N"],2))
    obj_stats.append(round( percent["D"],2))
    obj_stats.append(round( percent["C"],2))
    obj_stats.append(row["Index"])
    obj_stats.append(row["Class"])
    obj_stats.append(row["Binary"])


    statistical.append(obj_stats)


df2 = pd.DataFrame(statistical, columns=['Weight', 'Aromaticity', 'Instability', \
                                         'Helix', 'Turn', 'Sheet', 'Extinction', \
                                         'Charge10', 'Charge7', 'Charge4', \
                                         'Isoelectric', 'GRAVY', 'Flexibility', \
                                         'AverageWeight', 'Tiny', 'Small', \
                                         'Aliphatic', 'Aromatic', 'NonPolar', \
                                         'Polar', 'Charged', 'Basic', 'Acidic', \
                                         'Ala', 'Arg', 'Asn', 'Asp', 'Cys', \
                                         'Index', "Class", "Binary"])

df = df.set_index("Index")
df

,Sequence,Class,Binary
Index,,,
5_0,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1
5_1,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1
5_2,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...,5,1
5_3,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...,5,1
5_4,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...,5,1
...,...,...,...
4_1995,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...,4,1
4_1996,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...,4,1
4_1997,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...,4,1


## Implementing types of padding

In [8]:
def pad_sequence(sequence, maxlen, padding='post'):
  num_padding = maxlen - len(sequence)
  padded_sequence = ""

  if padding == 'post':
    padded_sequence = sequence + "0" * (maxlen - len(sequence))

  elif padding == 'extreme':
    half_padding = num_padding // 2
    padded_sequence = "0" * half_padding + sequence + "0" * (num_padding - half_padding)

  elif padding == 'mid':
    half_sequence = len(sequence) // 2
    padded_sequence = sequence[:half_sequence] + "0" * num_padding + sequence[half_sequence:]
            
  elif padding == 'uniform':
    for i, c in enumerate(sequence[:-1]):
        padded_sequence += c + "0"
        # If there are no more padding characters left, stop interleaving
        if i + 1 == num_padding:
            padded_sequence += sequence[i+1:]
            break
    # If there are still some padding characters left, add them to the end of the string
    padded_sequence += "0" * (num_padding - len(padded_sequence) + len(sequence))

  return padded_sequence

In [9]:
# Test the function
print(pad_sequence("abcd", 10,"post")) 
print(pad_sequence("abcd", 10,"extreme"))  
print(pad_sequence("abcd", 10,"mid"))
print(pad_sequence("abcd", 10,"uniform")) 

abcd000000
000abcd000
ab000000cd
a0b0c00000


In [10]:
# Test the function
print(pad_sequence("abcdefg", 10,"post"))  
print(pad_sequence("abcdefg", 10,"extreme"))  
print(pad_sequence("abcdefg", 10,"mid"))
print(pad_sequence("abcdefg", 10,"uniform"))

abcdefg000
0abcdefg00
abc000defg
a0b0c0defg


# First stage - perform task 1 and task 2 on the first Dataset with different types of paddings 

In [11]:
df['post'] = df['Sequence'].apply(lambda x: pad_sequence(x, 1000,"post"))

In [12]:
df['extr'] = df['Sequence'].apply(lambda x: pad_sequence(x, 1000,"extreme"))


In [13]:
df['mid']  = df['Sequence'].apply(lambda x: pad_sequence(x, 1000,"mid"))


In [14]:
df

,Sequence,Class,Binary,post,extr,mid
Index,,,,,,
5_0,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...,0000000000000000000000000000000000000000000000...,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...
5_1,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...,0000000000000000000000000000000000000000000000...,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...
5_2,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...,5,1,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...,0000000000000000000000000000000000000000000000...,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...
5_3,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...,5,1,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...,0000000000000000000000000000000000000000000000...,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...
5_4,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...,5,1,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...,0000000000000000000000000000000000000000000000...,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...
...,...,...,...,...,...,...
4_1995,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...,4,1,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...,000000000000MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKE...,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...
4_1996,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...,4,1,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...,0000000000000000000000000000000000000000000000...,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...
4_1997,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...,4,1,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...,0000000000000000000000000000000000000000000000...,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...


## Implementing pI encoding and scaling physico-chemical data

The project takes two approaches for binary and multiclass classification. 

The first approach is to use sequence data and represent it as an array of pIs corresponding to each aminoacid, for example:

QGHEAA = [-1.35, -1.03, 0.59, -3.78, -0.98, -0.98]

In the second approach, ProtParam is run on an input sequence and 28 physico-chemial parameters are extracted

AA....pI........pi - 7 \
A  -> 6.02  -> -0.98 \
R  -> 10.76 ->  3.76 \
N  -> 5.41  -> -1.59 \
D  -> 2.98  -> -4.02 \
C  -> 5.02 ->  -1.98 \
E  -> 3.22  -> -3.78 \
Q  -> 5.65  -> -1.35 \
G  -> 5.97 ->  -1.03 \
H  -> 7.59  ->  0.59 \
I  -> 5.98  -> -1.02 \
L  -> 5.98 ->  -1.02 \
K  -> 9.87  ->  2.87 \
M  -> 5.75  -> -1.25 \
F  -> 5.48 ->  -1.52 \
P  -> 6.30  -> -0.70 \
S  -> 5.68  -> -1.32 \
T  -> 5.60  -> -1.40 \
W  -> 5.94  -> -1.06 \
Y  -> 5.66  -> -1.34 \
V  -> 5.97  -> -1.03 \

In [15]:
def isoelectric_encoding(sequence):
  alphabet_dict = {'0': 0, 'A': -0.98, 'C': -1.98, 'D': -4.02, 'E': -3.78, \
                   'F': -1.52, 'G': -1.03, 'H': 0.59, 'I': -1.02 ,'K':2.87,\
                   'L': -1.02, 'M': -1.25 , 'N':-1.59 , 'P':-0.70, 'Q':-1.35, \
                   'R': 3.76, 'S':-1.32, 'T':-1.40, 'V':-1.03, 'W':-1.06, 'Y':-1.34}

  vector = np.array([alphabet_dict[char] for char in sequence])
  return vector

In [16]:
df['post'] = df['post'].apply(lambda x: isoelectric_encoding(x))

In [17]:
df['extr'] = df['extr'].apply(lambda x: isoelectric_encoding(x))


In [18]:
df['mid']  = df['mid'].apply(lambda x: isoelectric_encoding(x))


In [19]:
df

,Sequence,Class,Binary,post,extr,mid
Index,,,,,,
5_0,MSGDHSHNEDQIMGGSRINDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1,"[-1.25, -1.32, -1.03, -4.02, 0.59, -1.32, 0.59...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.25, -1.32, -1.03, -4.02, 0.59, -1.32, 0.59..."
5_1,MSGDHSHNEDQIMGGSRNNDSHKHKDKYKEHKHKDYKRDKEREKSK...,5,1,"[-1.25, -1.32, -1.03, -4.02, 0.59, -1.32, 0.59...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.25, -1.32, -1.03, -4.02, 0.59, -1.32, 0.59..."
5_2,MVGAAFVRPKRSPQRGDGVGRMLGGSRAVGTCGYGVNMRCLAARVN...,5,1,"[-1.25, -1.03, -1.03, -0.98, -0.98, -1.52, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.25, -1.03, -1.03, -0.98, -0.98, -1.52, -1...."
5_3,VGEGCDVPCLRRRGGPYKTAAATDLSRWRLSNVEGRQSWSFVDQRD...,5,1,"[-1.03, -1.03, -3.78, -1.03, -1.98, -4.02, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.03, -1.03, -3.78, -1.03, -1.98, -4.02, -1...."
5_4,MTEGTHLRRRGGPYKSDPATDLSRWRLTNDEGRQTWRYVEDQDSPD...,5,1,"[-1.25, -1.4, -3.78, -1.03, -1.4, 0.59, -1.02,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.25, -1.4, -3.78, -1.03, -1.4, 0.59, -1.02,..."
...,...,...,...,...,...,...
4_1995,MAGRVPSLLVLLLVFPSSCLAFRSPLSVFKRFKETTRPLSNECLGT...,4,1,"[-1.25, -0.98, -1.03, 3.76, -1.03, -0.7, -1.32...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.25, -0.98, -1.03, 3.76, -1.03, -0.7, -1.32..."
4_1996,MNLPPCRLWRPLTSRLGQRQPQPRAGARSCPLPARGRARPLCGRPH...,4,1,"[-1.25, -1.59, -1.02, -0.7, -0.7, -1.98, 3.76,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.25, -1.59, -1.02, -0.7, -0.7, -1.98, 3.76,..."
4_1997,NVSMDTWNFSYTCLVSLWLHRFYIYSVVAFGISVWIIIQFFTTKTK...,4,1,"[-1.59, -1.03, -1.32, -1.25, -4.02, -1.4, -1.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.59, -1.03, -1.32, -1.25, -4.02, -1.4, -1.0..."


In [20]:
df2

,Weight,Aromaticity,Instability,Helix,Turn,Sheet,Extinction,Charge10,Charge7,Charge4,...,Basic,Acidic,Ala,Arg,Asn,Asp,Cys,Index,Class,Binary
0,89099.48,0.09,43.92,0.23,0.17,0.24,110240,-49.72,36.30,144.24,...,0.26,0.18,0.05,0.06,0.04,0.08,0.01,5_0,5,1
1,84186.65,0.09,42.74,0.23,0.16,0.24,104740,-52.26,31.22,135.93,...,0.26,0.19,0.06,0.06,0.04,0.08,0.01,5_1,5,1
2,71911.16,0.11,34.71,0.31,0.23,0.24,106690,-27.76,16.72,72.44,...,0.15,0.10,0.08,0.06,0.03,0.04,0.01,5_2,5,1
3,80054.88,0.11,45.18,0.31,0.22,0.25,170740,-56.67,-9.42,57.32,...,0.12,0.10,0.08,0.06,0.03,0.06,0.04,5_3,5,1
4,82745.08,0.11,43.43,0.31,0.23,0.25,170170,-70.45,-17.51,58.33,...,0.12,0.12,0.07,0.05,0.03,0.06,0.03,5_4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,108712.95,0.09,47.60,0.30,0.26,0.22,95230,-77.11,-19.34,92.95,...,0.14,0.13,0.05,0.05,0.04,0.06,0.02,4_1995,4,1
27996,93586.65,0.07,35.46,0.28,0.25,0.23,98780,-41.52,14.79,96.57,...,0.15,0.11,0.08,0.06,0.04,0.06,0.02,4_1996,4,1
27997,82303.75,0.11,36.67,0.30,0.21,0.26,147250,-57.94,4.67,84.39,...,0.16,0.12,0.08,0.05,0.04,0.05,0.03,4_1997,4,1
27998,77531.96,0.08,43.55,0.31,0.23,0.25,41830,-43.59,7.98,77.52,...,0.15,0.12,0.06,0.06,0.02,0.05,0.03,4_1998,4,1


## Scale the table with physicochemical parameters

In [21]:
scaler = StandardScaler()
data_subset = df2.iloc[:,0:28]
scaled_subset = scaler.fit_transform(data_subset)
scaled_df = pd.DataFrame(scaled_subset,columns=data_subset.columns)
scaled_df

,Weight,Aromaticity,Instability,Helix,Turn,Sheet,Extinction,Charge10,Charge7,Charge4,...,NonPolar,Polar,Charged,Basic,Acidic,Ala,Arg,Asn,Asp,Cys
0,0.122565,0.428075,-0.164488,-1.582710,-1.965963,-0.549310,0.717823,0.379109,2.309880,3.596773,...,-2.885270,2.885163,4.301177,4.936276,2.432950,-1.062543,0.415398,0.136213,2.265319,-1.004342
1,-0.261003,0.428075,-0.299549,-1.582710,-2.255166,-0.549310,0.549156,0.254213,2.023867,3.166481,...,-3.322498,3.322370,4.301177,4.936276,2.836802,-0.512877,0.415398,0.136213,2.265319,-1.004342
2,-1.219409,1.423019,-1.218652,0.467397,-0.230743,-0.549310,0.608956,1.458918,1.207491,-0.121035,...,0.393943,-0.393884,-0.179154,0.494987,-0.797868,0.586454,0.415398,-0.704607,-0.946964,-1.004342
3,-0.583590,1.423019,-0.020270,0.467397,-0.519946,-0.227457,2.573166,0.037367,-0.264239,-0.903950,...,0.831171,-0.831090,-0.886575,-0.716273,-0.797868,0.586454,0.415398,-0.704607,0.659178,1.447409
4,-0.373554,1.423019,-0.220573,0.467397,-0.230743,-0.227457,2.555686,-0.640218,-0.719721,-0.851652,...,0.612557,-0.612487,-0.414961,-0.716273,0.009837,0.036788,-0.292523,-0.704607,0.659178,0.630158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,1.653882,0.428075,0.256720,0.211133,0.636867,-1.193014,0.257514,-0.967701,-0.822753,0.940974,...,-0.043286,0.043322,0.292459,0.091234,0.413689,-1.062543,-0.292523,0.136213,0.659178,-0.187092
27996,0.472899,-0.566870,-1.132808,-0.301393,0.347664,-0.871162,0.366381,0.782317,1.098828,1.128418,...,0.393943,-0.393884,0.056653,0.494987,-0.394016,0.586454,0.415398,0.136213,0.659178,-0.187092
27997,-0.408010,1.423019,-0.994313,0.211133,-0.809150,0.094395,1.852803,-0.025081,0.529054,0.497736,...,-0.043286,0.043322,0.528266,0.898741,0.009837,0.586454,-0.292523,0.136213,-0.143893,0.630158
27998,-0.780567,-0.069397,-0.206838,0.467397,-0.230743,-0.227457,-1.380095,0.680531,0.715413,0.142007,...,0.393943,-0.393884,0.292459,0.494987,0.009837,-0.512877,0.415398,-1.545427,-0.143893,0.630158


### The project experiments on three model architectures, with no convolution, with one convolutional layer and with three convolutional layers (implemented from scratch in Keras)

## Architecture #1 - no convolution

In [22]:
def noConv(input_size, hidden_size, num_classes):
    inputs = Input(shape=(input_size,))
    fc1 = Dense(hidden_size, activation='relu')(inputs)
    dropout1 = Dropout(0.5)(fc1)
    fc2 = Dense(hidden_size, activation='relu')(dropout1)
    dropout2 = Dropout(0.25)(fc2)
    fc3 = Dense(num_classes, activation='sigmoid')(dropout2)
    model = Model(inputs, fc3)
    return model




## Architecture #2 - 1 convolutional layer

In [71]:
def oneCNN(input_size, num_classes):

    inputs = Input(shape=(input_size, 1))
    conv_layer1 = Conv1D(32, 3, activation='relu', input_shape=(input_size, 1))(inputs)
    pooling_layer1 = MaxPooling1D(pool_size=2)(conv_layer1)
    dropout1 = Dropout(0.5)(pooling_layer1)
    flatten = Flatten()(conv_layer1)
    fc1 = Dense(16, activation='relu')(flatten)
    fc2 = Dense(8, activation='relu')(fc1)
    fc3 = Dense(num_classes, activation='sigmoid')(fc2)
    
    model = Model(inputs, fc3)
    return model


## Architecture #3 - stack of 5 convolutional layers

In [72]:
def stackedCNN(input_size, num_classes):
    inputs = Input(shape=(input_size, 1))
    conv_layer1 = Conv1D(32, 2, activation='relu', input_shape=(input_size, 1))(inputs)
    pooling_layer1 = MaxPooling1D(pool_size=2)(conv_layer1)
    conv_layer2 = Conv1D(256, 2, activation='relu')(pooling_layer1)
    dropout1 = Dropout(0.5)(conv_layer2)
    conv_layer3 = Conv1D(128, 2, activation='relu')(dropout1)
    pooling_layer2 = MaxPooling1D(pool_size=2)(conv_layer3)
    conv_layer4 = Conv1D(64, 2, activation='relu')(pooling_layer2)
    dropout2 = Dropout(0.25)(conv_layer4)
    conv_layer5 = Conv1D(32, 2, activation='relu')(dropout2)
    flatten = Flatten()(conv_layer5)
    fc1 = Dense(16, activation='relu')(flatten)
    fc2 = Dense(8, activation='relu')(fc1)
    fc3 = Dense(num_classes, activation='sigmoid')(fc2)
    
    model = Model(inputs, fc3)
    return model


## Test different kinds of paddings on dataset 1


As a first stage of the project, post, extreme and mid padding were used in task1 - binary classification and in task2 - multiclass classification on dataset 1.

Task 1 - binary classification 14000 enzymes and 14000 non-enzymes

Hiperparameters - batch_size and epochs were manually tested, 64 and 45 were chosen. 

In [69]:
batch_size = 64
epochs = 45

In [ ]:
for padding in ["post", "extr", "mid"]:
  X = df[padding].values
  X = X.tolist()
  X = np.stack(X)
  y = df2['Binary'].values
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
  X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

  model = noConv(1000, 64, 1)
  model.compile(optimizer=Adam(), loss = binary_crossentropy, metrics=["accuracy"])
  model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, verbose=1)
  _, test_acc = model.evaluate(X_test, y_test, verbose=0)
  print("Test accuracy:", test_acc)
  model.summary()



Epoch 1/45
350/350 [==============================] - 1s 3ms/step - loss: 0.6924 - accuracy: 0.5600
Epoch 2/45
350/350 [==============================] - 1s 3ms/step - loss: 0.6304 - accuracy: 0.6165
Epoch 3/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5942 - accuracy: 0.6554
Epoch 4/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5746 - accuracy: 0.6749
Epoch 5/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5531 - accuracy: 0.6908
Epoch 6/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5388 - accuracy: 0.7040
Epoch 7/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5195 - accuracy: 0.7142
Epoch 8/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5044 - accuracy: 0.7252
Epoch 9/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4948 - accuracy: 0.7331
Epoch 10/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4831 - accuracy: 0.7425

The paddings got the following accuracy:



1.   post - 60.5%
2.   extreme - 64.6%
3.   mid - 70.3%



Testing out task 2 - 14000 enzymes in 7 classes per 2000 observations (multiclass classification)

In [26]:
df3 = df[df["Binary"] != 0]
df4 = df2[df2["Binary"] != 0]
data_subset = df4.iloc[:,0:28]
scaled_subset = scaler.fit_transform(data_subset)
scaled_df2 = pd.DataFrame(scaled_subset,columns=data_subset.columns)
scaled_df2

,Weight,Aromaticity,Instability,Helix,Turn,Sheet,Extinction,Charge10,Charge7,Charge4,...,NonPolar,Polar,Charged,Basic,Acidic,Ala,Arg,Asn,Asp,Cys
0,0.019064,0.340273,0.184694,-2.247021,-2.365072,-0.641198,0.684356,0.387380,2.649837,3.546482,...,-3.561624,3.561163,4.386968,4.699078,2.991882,-1.275096,0.502404,0.274225,2.582006,-1.256611
1,-0.354484,0.340273,0.020202,-2.247021,-2.756400,-0.641198,0.504808,0.227563,2.296024,3.107736,...,-4.062218,4.061678,4.386968,4.699078,3.468611,-0.642608,0.502404,0.274225,2.582006,-1.256611
2,-1.287853,1.426167,-1.099181,0.267505,-0.017107,-0.641198,0.568465,1.769109,1.286124,-0.244375,...,0.192836,-0.192698,-0.146362,0.436173,-0.821949,0.622368,0.502404,-0.700179,-1.076892,-1.256611
3,-0.668644,1.426167,0.360338,0.267505,-0.408434,-0.253684,2.659384,-0.049915,-0.534483,-1.042672,...,0.693431,-0.693213,-0.862151,-0.726438,-0.821949,0.622368,0.502404,-0.700179,0.752557,1.934685
4,-0.464094,1.426167,0.116388,0.267505,-0.017107,-0.253684,2.640776,-0.916956,-1.097938,-0.989347,...,0.443134,-0.442956,-0.384958,-0.726438,0.131509,-0.010120,-0.247213,-0.700179,0.752557,0.870920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,1.510377,0.340273,0.697687,-0.046811,1.156876,-1.416227,0.194353,-1.336005,-1.225394,0.838501,...,-0.307758,0.307817,0.330831,0.048636,0.608238,-1.275096,-0.247213,0.274225,0.752557,-0.192845
13996,0.360246,-0.745621,-0.994631,-0.675442,0.765549,-1.028713,0.310243,0.903326,1.151702,1.029628,...,0.192836,-0.192698,0.092235,0.436173,-0.345220,0.622368,0.502404,0.274225,0.752557,-0.192845
13997,-0.497651,1.426167,-0.825957,-0.046811,-0.799762,0.133831,1.892551,-0.129824,0.446862,0.386555,...,-0.307758,0.307817,0.569427,0.823710,0.131509,0.622368,-0.247213,0.274225,-0.162168,0.870920
13998,-0.860474,-0.202674,0.133116,0.267505,-0.017107,-0.253684,-1.548895,0.773081,0.677398,0.023837,...,0.192836,-0.192698,0.330831,0.436173,0.131509,-0.642608,0.502404,-1.674583,-0.162168,0.870920


In [27]:
df3 = pd.get_dummies(df3, columns=['Class'])
df4 = pd.get_dummies(df4, columns=['Class'])


In [ ]:
for padding in ["post", "extr", "mid"]:
  X = df3[padding].values
  X = X.tolist()
  X = np.stack(X)
  y = df3.loc[:,'Class_1':'Class_7'].values

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
  X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


  model = noConv(1000, 64, 7)
  model.compile(optimizer=Adam(), loss = categorical_crossentropy, metrics=["accuracy"])
  model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, verbose=1)
  _, test_acc = model.evaluate(X_test, y_test, verbose=0)
  print("Test accuracy:", test_acc)
  model.summary()



Epoch 1/45
175/175 [==============================] - 2s 5ms/step - loss: 1.9051 - accuracy: 0.2301
Epoch 2/45
175/175 [==============================] - 1s 5ms/step - loss: 1.5891 - accuracy: 0.3604
Epoch 3/45
175/175 [==============================] - 0s 3ms/step - loss: 1.4079 - accuracy: 0.4341
Epoch 4/45
175/175 [==============================] - 0s 3ms/step - loss: 1.3063 - accuracy: 0.4714
Epoch 5/45
175/175 [==============================] - 0s 3ms/step - loss: 1.1953 - accuracy: 0.5273
Epoch 6/45
175/175 [==============================] - 0s 3ms/step - loss: 1.1285 - accuracy: 0.5566
Epoch 7/45
175/175 [==============================] - 0s 3ms/step - loss: 1.0588 - accuracy: 0.5798
Epoch 8/45
175/175 [==============================] - 0s 3ms/step - loss: 1.0181 - accuracy: 0.5954
Epoch 9/45
175/175 [==============================] - 0s 3ms/step - loss: 0.9822 - accuracy: 0.6098
Epoch 10/45
175/175 [==============================] - 0s 3ms/step - loss: 0.9328 - accuracy: 0.6281

The paddings got the following accuracy:

1.   post - 67.4%
2.   extreme - 62.3%
3.   mid - 80.2%

Middle padding clearly got the best accuracy within the given dataset and tested model.

In the next part i will use three architectures:


*   no convolution (Architecture #1 already tested here)
*   one convolutional layer (Architecture #2)
*   five convolutional layers (Architecture #3)


I will also use the physico-chemical tabular data comprised in scaled_df to the same tasks with the same architectures.

The analysis will also be done for three separate datasets: \

Dataset 1: 14000 enzymes of 7 classes, 14000 non-enzymes. (this notebook)\
Dataset 2: 5000 enzymes from 5 specific subclasses and 5000 non-enzymes. (notebook 2) \
Dataset 3: task 2: 770 proteins from cytoplasm, membrane and extracellular space classified to 3 classes based on their location.



Prepare Data

In [29]:
def split_data(X,y):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
  X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

  return X_train, X_test, y_train, y_test

In [30]:
# task1_sequence
X = df["mid"].values
X = X.tolist()
X = np.stack(X)
y = df['Binary'].values

X_train, X_test, y_train, y_test = split_data(X,y)

#task1_tabular
X2 = scaled_df.values
y2 = df2['Binary'].values

X_train2, X_test2, y_train2, y_test2 = split_data(X2,y2)

#task2_sequence
X3 = df3["mid"].values
X3 = X3.tolist()
X3 = np.stack(X3)
y3 = df3.loc[:,'Class_1':'Class_7'].values


X_train3, X_test3, y_train3, y_test3 = split_data(X3,y3)


#task2_tabular
X4 = scaled_df2.values
y4 = df4.loc[:,'Class_1':'Class_7'].values

X_train4, X_test4, y_train4, y_test4 = split_data(X4,y4)



In [31]:
input_tabular = 28
input_sequence = 1000
hidden_size = 64

In [67]:
def run_model(architecture, tabular, out_classes, X_train, X_test, y_train, y_test):

    if architecture == "no_conv":
      if tabular == True:
        model = noConv(input_tabular, hidden_size, out_classes)
      elif tabular == False:
        model = noConv(input_sequence, hidden_size, out_classes)

    if architecture == "one_conv":
      if tabular == True:
        model = oneCNN(input_tabular, out_classes)
      elif tabular == False:
        model = oneCNN(input_sequence, out_classes)

    if architecture == "five_conv":
      if tabular == True:
        model = stackedCNN(input_tabular, out_classes)
      elif tabular == False:
        model = stackedCNN(input_sequence, out_classes)

    if out_classes == 1:
      loss = binary_crossentropy
    else:  
      loss = categorical_crossentropy

    model.compile(optimizer=Adam(), loss = loss, metrics=["accuracy"])
    model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, verbose=1)
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(str(out_classes) + " " + str(tabular) + " " + architecture)
    print("Test accuracy:", test_acc)
    model.summary()

    return model

Rewriting this as a loop takes the same amount of space and is less readable:

In [73]:
model = run_model("no_conv", False, 1, X_train, X_test, y_train, y_test)
model = run_model("no_conv", True,  1, X_train2, X_test2, y_train2, y_test2)
model = run_model("no_conv", False, 7, X_train3, X_test3, y_train3, y_test3)
model = run_model("no_conv", True,  7, X_train4, X_test4, y_train4, y_test4)

model = run_model("one_conv", False, 1, X_train, X_test, y_train, y_test)
model = run_model("one_conv", True,  1, X_train2, X_test2, y_train2, y_test2)
model = run_model("one_conv", False, 7, X_train3, X_test3, y_train3, y_test3)
model = run_model("one_conv", True,  7, X_train4, X_test4, y_train4, y_test4)

model = run_model("five_conv", False, 1, X_train, X_test, y_train, y_test)      #model1
model = run_model("five_conv", True,  1, X_train2, X_test2, y_train2, y_test2)  #model2
model = run_model("five_conv", False, 7, X_train3, X_test3, y_train3, y_test3)  #model3  
model = run_model("five_conv", True,  7, X_train4, X_test4, y_train4, y_test4)  #model4



Epoch 1/45
350/350 [==============================] - 1s 3ms/step - loss: 0.6622 - accuracy: 0.6065
Epoch 2/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5665 - accuracy: 0.6813
Epoch 3/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5322 - accuracy: 0.7092
Epoch 4/45
350/350 [==============================] - 1s 3ms/step - loss: 0.5053 - accuracy: 0.7283
Epoch 5/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4807 - accuracy: 0.7440
Epoch 6/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4647 - accuracy: 0.7543
Epoch 7/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4533 - accuracy: 0.7622
Epoch 8/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4392 - accuracy: 0.7690
Epoch 9/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4331 - accuracy: 0.7729
Epoch 10/45
350/350 [==============================] - 1s 3ms/step - loss: 0.4215 - accuracy: 0.7794

## combine best models for task1

In [ ]:
def combine_binary(m1, m2, X_1, X_2, y_t):
  logits1 = m1(X_1)
  logits2 = m2(X_2)
  logits = (logits1 + logits2) / 2
  y_pred = (logits > 0.5)
  arr = np.where(y_pred, 1, 0)
  acc = accuracy_score(y_t, arr)
  return acc

In [ ]:
print(combine_binary(model1, model2, X_test, X_test2, y_test))

0.7382142857142857


## combine best models for task2

In [ ]:
def combine_multiclass(m1, m2, X_1, X_2, y_t):
  logits1 = m1(X_1)
  logits2 = m2(X_2)
  logits = (logits1 + logits2) / 2
  arr = tf.keras.backend.get_value(logits)
  y_pred = np.eye(arr.shape[1])[np.argmax(arr, axis=1)]
  acc = accuracy_score(y_t, y_pred)
  return acc

0.8992857142857142


In [ ]:
print(combine_multiclass(model3, model4, X_test3, X_test4, y_test3))

0.8992857142857142
